In [117]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse

In [129]:
base_url = "https://codigo-postal.co/argentina/capital-federal/calles-de-ciudad-autonoma-buenos-aires/"
url = "https://codigo-postal.co/argentina/capital-federal/ciudad-autonoma-buenos-aires"
response = requests.get(base_url)

https://codigo-postal.co/argentina/capital-federal/ciudad-autonoma-buenos-aires


In [119]:
# Verificar el estado de la respuesta
if response.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")
    
    
    # Encontrar la tabla que contiene las localidades de Córdoba
    table = soup.find("ul", class_="three_columns")
    # Encontrar todos los elementos <a> dentro del elemento <ul>
    a_elements = table.find_all("a")
    
    #  Obtener el texto de cada elemento <a>
    localidades = [a.text for a in a_elements]
    
    # Crear un DataFrame con las localidades
    df_localidades = pd.DataFrame({"Localidad": localidades})
    
    # Guardar el DataFrame en un archivo CSV
    df_localidades.to_csv("localidades_capital_federal.csv", index=False)
    
    print("Archivo CSV guardado exitosamente.")
else:
    print("Error al acceder a la página de Capital Federal")

Archivo CSV guardado exitosamente.


In [139]:
# Leer el archivo CSV de localidades
df_localidades = pd.read_csv("localidades_capital_federal.csv")

# Reemplazar espacios por guiones y convertir a minúsculas en cada localidad
df_localidades["Localidad"] = df_localidades["Localidad"].str.replace(" ", "-").str.lower()
df_localidades["Localidad"] = df_localidades["Localidad"].str.split("-").str[1:].str.join("-")

# Crear una lista para almacenar los datos de los CPAs
data = []

base_url = url  # Reemplaza "ejemplo.com" con la URL base correcta

for localidad in df_localidades["Localidad"]:
    localidad_codificada = urllib.parse.quote(localidad)
    url_localidad = f"{base_url}/{localidad_codificada}"
    
    # Realizar solicitud GET sin redireccionamiento automático
    response = requests.get(url_localidad, allow_redirects=True)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Obtener los textos de los elementos <a>
        a_texts = []
        tbody = soup.find("tbody")
        
        if tbody:
            rows = tbody.find_all("tr")
            
            for row in rows:
                cells = row.find_all("td")
                
                for cell in cells:
                    a_elements = cell.find_all("a")
                    
                    for a in a_elements:
                        a_texts.append(a.text)
        
        # Agregar los datos a la lista
        data.append({"Localidad": localidad, "CPAs": a_texts})
        
    else:
        print(f"Error al acceder a la página de {localidad}")

# Crear un DataFrame a partir de los datos recolectados
df_resultado = pd.DataFrame(data)

# Guardar el DataFrame en un archivo CSV
df_resultado.to_csv("capital_federal_cpa.csv", index=False)
print("Archivo CSV guardado exitosamente.")

TooManyRedirects: Exceeded 30 redirects.